### Initialization

In [1]:
import glob
import json
import re
from nltk import tokenize
import textdistance
import random
import string
import os
import numpy
import pandas
import fasttext
import gensim
import conllu
import editdistance

In [2]:
from utils.text_extraction import *
from utils.document_processing import *
from utils.subject_extraction import *
from utils.similarity import *
from utils.subject_context import *

In [3]:
import psycopg2
psycopg2_conn = psycopg2.connect(dbname='public_contracts', user='postgres', password='admin', host='localhost', port='5432')

In [4]:
from ufal.udpipe import *
# load model from the given file;
# if the file does not exist, expect a Segmentation fault
udpipe_model = Model.load("udpipe/udpipe-ud-2.4-190531/czech-pdt-ud-2.4-190531.udpipe")

### Subject context

In [5]:
from utils.subject_context import *

### Subject extraction

In [6]:
from utils.subject_extraction import *

### EvaluationMachine

In [7]:
class EvaluationMachine:
    
    _core = None
    _subjects = None
    _ref_subjects = None
    _similarity_matrix = None
    
    def __init__(self, core=None):
        self._core = core
    
    def evaluateSubjects(self, subjects, ref_subjects):
        self._subjects = [' '.join(s) if isinstance(s, list) else s for s in subjects]
        self._ref_subjects = [' '.join(s) if isinstance(s, list) else s for s in ref_subjects]
        return self.evaluate()
    
    def evaluate(self):
        self.compute_matrix()
        return self._similarity_matrix
        
    def compute_matrix(self):
        sl = len(self._subjects)
        rl = len(self._ref_subjects)
        result_matrix = numpy.zeros((sl, rl))
        for i in range(sl):
            for j in range(rl):
                d = self._core.compute(self._subjects[i], self._ref_subjects[j])
                result_matrix[i][j] = d
        self._similarity_matrix = result_matrix
        
    def display(self, with_headers=False):
        if with_headers:
            df = pandas.DataFrame(self._similarity_matrix, columns=self._ref_subjects, index=self._subjects)
        else:
            df = pandas.DataFrame(self._similarity_matrix)
        display(df)
        
    
class RandomEvaluationMachine(EvaluationMachine):
    
    def __init__(self, core=None):
        super().__init__(RandomSimilarityMachine())
        
class JaccardEvaluationMachine(EvaluationMachine):

    def __init__(self, core=None):
        super().__init__(JaccardSimilarityMachine())
        
class FastTextEvaluationMachine(EvaluationMachine):
    
    def __init__(self, core=None):
        super().__init__(FastTextSimilarityMachine(core))
        
class GensimFastTextEvaluationMachine(EvaluationMachine):
    
    def __init__(self, core=None):
        super().__init__(GensimFastTextSimilarityMachine(core))

In [24]:
class SubjectExtractor():
    
    _keywords = None
    _subj_range = None
    
    def __init__(self, keywords={"Předmět smlouvy":1}, subj_range=2000):
        self._keywords = keywords
        self._subj_range = subj_range
        
    def get_all_occurrences(self, text):
        occurrences = []
        for keyword in self._keywords:
            occ = find_all_occurences_in_string(keyword, text)
            for o in occ:
                rat = self._keywords[keyword]
                koef = 1
                matched = keyword.lower()
                # Exact pattern match
                if text[o:min(o+len(keyword),len(text))] == keyword:
                    koef+=1.5
                    matched = keyword
                # Upper case pattern match
                if text[o:min(o+len(keyword),len(text))] == keyword.upper():
                    koef+=1.5
                    matched = keyword.upper()
                # Nearly linebreak after the pattern (chapter title)
                if '\n' in text[o:min(o+len(keyword)*3,len(text))]:
                    koef+=2
                # Newline followed by a number preceding the pattern (chapter numbering)
                if re.search(r"\n[ ]*[0-9]", text[max(o-20,0):o]):
                    koef+=2
                # Nearly verb ' je ' after the pattern (subject sentence matching)
                if ' je ' in text[o:min(o+len(keyword)*2,len(text))]:
                    koef+=2
                # Word 'článek' preceding the pattern (chapter header)
                if 'článek' in text[max(o-20,0):o].lower():
                    koef+=2
                # Chars 'I' preceding the pattern (chapter numbering)
                if text[max(o-20,0):o].count('I')>1:
                    koef+=2
                rat*=koef
                occurrences.append({'keyword':matched, 'rat': rat, 'occ':o})
        return occurrences
    
    def extract(self, text):
        occurrences = self.get_all_occurrences(text)
        df = pandas.DataFrame(occurrences, columns=['keyword', 'rat', 'occ'])        
        df = df.sort_values('rat', ascending=False)
        if len(df.index)>0:
            best_hit = df.iloc[0]['occ']
        else:
            best_hit = 0
        
        start = best_hit
        end = min(start + self._subj_range, len(text))
        subj_context = text[start:end]
        return subj_context
  
    def show_occurrence_distribution(self, text):
        bins = 100
        bin_size = int(self._subj_range/bins)
        text = text.rjust(bins*bin_size)

        occurrences = self.get_all_occurrences(text)
        df = pandas.DataFrame(occurrences)
        hist = numpy.histogram(df['occ'], bins, weights=df['rat'], range=(0,len(text)))
        pandas.DataFrame(hist[0]).plot.bar()

In [25]:
class SubjectContextCopyExtractor(SubjectExtractor):
    
    def __init__(self, param=None):
        pass
        
    def extract(self, text):
        return text

In [26]:
class SubjectExtractionEvaluationMachine():
    
    _path_to_data = None
    _df_contracts = None
    _subj_extractor = None
    _func_param = None
    
    def __init__(self, path_to_data, df_contracts, subj_extractor=SubjectContextCopyExtractor, func_params=None):
        self._df_contracts = df_contracts
        self._subj_extractor = subj_extractor
        self._path_to_data = path_to_data
        self._func_params = func_params

    def process(self):
        self._df_contracts['subject'] = self._df_contracts['subj_context'].apply(
            lambda subj_context: self._subj_extractor(**self._func_params).extract(subj_context))
        self._df_contracts = validate_subjects(self._df_contracts, self._path_to_data)
        return self._df_contracts
        
    def evaluate(self):
        return self._df_contracts['valid_rat'].mean()*100

### Similarity functions

In [27]:
from utils.similarity import *

### Documents 

In [28]:
from utils.document_processing import *

In [29]:
loader = DatabaseDocumentLoader(psycopg2_conn)
loader.load_documents()
documents = loader.prepare_documents()

Running query: select * from document where processed=True
Preparing total 998 documents
Progress: 0.0%
Progress: 10.0%
Progress: 20.0%
Progress: 30.0%
Progress: 40.0%
Progress: 50.0%
Progress: 60.0%
Progress: 70.0%
Progress: 80.0%
Progress: 90.0%
Progress: 100.0%


In [30]:
vzdirs = '../test-data/*/*/*/*'
ignore_contracts = ['2096','sportovni-areal-zs-slatina-1-etapa_25534',
                    'revitalizace-lokality-spaliste-k-u-stare-mesto-u-uherskeho-hradiste-akceptacni-cislo-12136894-cz-1-02-4-2-00-12-16399-dodatecne-prace_9496',
                    '35226', 'kompostujeme-v-obci-sebranice-dodavka-stepkovace-a-komposteru', 'zvysovani-povedomi-verejnosti-o-biodiverzite-luk-a-pastvin',
                    'nakup-noveho-uzitkoveho-elektromobilu-typu-bev-vozidlo-s-bateriovym-pohonem']

ref_loader = ReferenceDocumentLoader(vzdirs)
# ref_loader.extract_text_from_documents(0,1024)
ref_loader.load_documents_from_extracts(ignore_contracts)
ref_documents = ref_loader.prepare_documents()

Loading total 316 extracts
Preparing total 316 documents
Progress: 0.0%
Progress: 10.0%
Progress: 20.0%
Progress: 30.0%
Progress: 40.0%
Progress: 50.0%
Progress: 59.0%
Progress: 69.0%
Progress: 79.0%
Progress: 89.0%
Progress: 99.0%


In [31]:
ignore_mask = count_occurence_vector('qwertyuiopasdfghjklzxcvbnmQWERTYUIOPASDFGHJKLZXCVBNMáčďéěíňóřšťúůýžÁČĎÉĚÍŇÓŘŠŤÚŮÝŽ0123456789')
ignore_mask[0]=0
matcher = DocumentMatcher(ref_documents, documents)
matcher.count_most_similar_documents(ignore_mask=ignore_mask)
matcher.aggregate_documents()
df_contract = matcher.filter_aggregated()

Counting most similar documents for total 316 reference documents
Progress: 0.0%
Progress: 10.0%
Progress: 20.0%
Progress: 30.0%
Progress: 40.0%
Progress: 50.0%
Progress: 59.0%
Progress: 69.0%
Progress: 79.0%
Progress: 89.0%
Progress: 99.0%


### Measurements

In [32]:
vzdirs = '../test-data/*/*/*'
def save_subjects(vzdirs, df_contracts):
    dirs = [path for path in glob.glob(vzdirs) if 'test_src' not in path]
    for path in dirs:
        contr_name = '/'.join(path.split('\\')[1:4])
        rows=df_contracts[df_contracts['contr_name']==contr_name]
        with open(path+'/_ref.json', encoding='utf-8') as f:
            data = f.read()
            ref_dict = json.loads(data)
        ref_dict['subject2']=rows.iloc[0]['ref_subj'] if len(rows)>0 else None
        with open(path+'/_ref.json', 'w', encoding='utf-8') as f:
            json.dump(ref_dict, f, ensure_ascii=False, indent=4)    

In [33]:
def validate_subjects(df_contracts, vzdirs):
    df_contracts['valid_rat']=0
    df_contracts['ref_subj']=None
    ref_paths = [path for path in glob.glob(vzdirs) if 'test_src' not in path]
    for path in ref_paths:
        contr_name = '/'.join(path.split('\\')[1:4])
        row = df_contracts[df_contracts.contr_name==contr_name]
        if len(row)>0:
            with open(path+'/_ref.json', 'r', encoding='utf8') as f:
                ref = f.read()
                ref_subj = ReferenceSubject2Extractor(path).extract()
            valid_rat = JaccardSimilarityMachine().compute(row.iloc[0]['subject'], ref_subj)
            df_contracts.valid_rat.loc[row.iloc[0].name] = valid_rat
            df_contracts.ref_subj.loc[row.iloc[0].name] = ref_subj        
    return df_contracts

In [34]:
contracts = []
for contr_name, docs in df_contract.groupby('contr_name'):
    contr_text = ''
    contr_ids = []
    for index, doc in docs.iterrows():
        doc_name = doc['doc_name']
        contr_text += '======================='+doc_name+'=======================\n'
        contr_text += doc['doc_text']
        contr_ids.append(doc['contr_id'])
    contr_id = get_most_frequent(contr_ids)
    contr = {'id': contr_id, 'contr_name': contr_name, 'text': contr_text}
    contracts.append(contr)

In [35]:
%%time 

vzdirs = '../test-data/*/*/*'

keywords = {
    'Předmět smlouvy':10,
    'Předmět díla':10,
    'Předmět plnění':10,
    'Předmět veřejné zakázky':10,
    'Vymezení předmětu':10,
    'Vymezení plnění': 10,
    'Název veřejné zakázky': 3,
    'Veřejná zakázka':1,
    'Veřejné zakázce':1,
    'Předmět': 1
   }
evaluator = ComplexSubjectContextSelectionEvaluationMachine(vzdirs, validator=validate_subj_contexts_v2, contracts=contracts,
                                                            subj_context_extractor=AdvancedSubjectContextExtractor,
                                                            func_params={'keywords': keywords, 'subj_range': 2000})
evaluator.process()
score = evaluator.evaluate()
print('Score: ' + str(score))
print('Improve: ' + str(score - last_score))
last_score = score
# df_last_contracts = df_contracts
df_contracts = evaluator._df_contracts
# scores3['w/o_capitals'] = score

Score: 74.79669084074135


NameError: name 'last_score' is not defined

In [878]:
save_valid_contexts(df_contracts, df_contract)

In [663]:
%%time 

vzdirs = '../test-data/*/*/*'

keywords = {
    'Předmět smlouvy':10,
    'Předmět díla':10,
    'Předmět plnění':10,
    'Předmět veřejné zakázky':10,
    'Vymezení předmětu':10,
    'Vymezení plnění': 10,
    'Název veřejné zakázky': 3,
    'Veřejná zakázka':1,
    'Veřejné zakázce':1,
    'Předmět': 1
   }
evaluator = SubjectExtractionEvaluationMachine(vzdirs, df_contracts, subj_extractor=SubjectExtractor,
                                               func_params={'keywords': keywords, 'subj_range':4000})
evaluator.process()
score = evaluator.evaluate()
print('Score: ' + str(score))
print('Improve: ' + str(score - last_score))
last_score = score
# scores2['"Předmět"_1'] = score

Score: 18.89384029423359
Improve: 0.0
Wall time: 226 ms


### Playground

In [36]:
df_contracts = evaluator._df_contracts

In [37]:
df_contracts.reset_index()

,id,contr_name,text,subj_context,valid_rat,ref_context,valid
0,71,e-zakazky/48a575ba-f8fe-47f2-bbb6-c6ddb26390ef...,=======================2017126-115451.pdf=====...,PŘEDMĚT SMLOUVY\nZhotovitel se zavazuje k prov...,0.733333,\nSmlouva nabývá platnosti a účinnosti dnem po...,True
1,72,e-zakazky/73b0be6b-6b49-4ab4-a6a0-508b83a305e1...,=======================Formulář - Informace o ...,PŘEDMĚT DÍLA\n\n1.1. Předmětem díla je provede...,0.985050,\nTito uvedení zástupci jsou oprávněni jednat ...,True
2,74,e-zakazky/c265fdf8-044f-44e5-b36e-df4ca53c6179...,=======================Dodatečné informace 01....,Předmět smlouvy\n1. Zhotovitel se touto smlouv...,0.471039,"3. Zhotovitel prohlašuje, že je odborně způsob...",NaN
3,61,eagri/mze/contract_display_6220,=======================01_výzva k podání nabíd...,Předmět smlouvy\n\nPředmětem smlouvy je proved...,0.975272,\n(dále jen „zhotovitel “ nebo „projektant“)\n...,True
4,64,ezak/mpsv/P15V00000447,=======================AKRIS_příloha-01_Kvalif...,Předmět veřejné zakázky a sjednaná cena ve sml...,0.576923,\nvyhotovil tuto písemnou zprávu.\n\n1. Předmě...,True
5,66,ezak/mpsv/P16V00000187,=======================Příloha č. 1 SOD_Projek...,"Předmět smlouvy, předmět díla\n\nPředmětem tét...",0.928721,\nprohlášení nebo v souvislosti s ní druhé sml...,True
6,67,ezak/mpsv/P16V00000240,=======================P1_Výkaz výměr (slepý)....,Předmět Smlouvy\n2.1. Zhotovitel se zavazuje p...,0.769231,\nmluvní straně vznikne.\nČlánek 2\nPředmět Sm...,True
7,103,nipez/mo/P18V00009525,=======================VZOR_smlouvy.doc=======...,Předmět smlouvy\n\n2.1. \nPředmětem smlouvy je...,0.993276,Článek 2\n\nPředmět smlouvy\n\n2.1. \nPředměte...,True
8,207,nipez/mo/P18V00012708,=======================Oznámení o výběru dodav...,Název veřejné zakázky: Sponkovače a spojovací ...,0.466667,\nIdentifikace veřejné zakázky\n\nNázev veřejn...,NaN
9,397,nipez/mo/P18V00019354,=======================PSP s přílohami.zip====...,Předmět smlouvy\n\n\n1. Poskytovatel se zavazu...,0.833333,nutného k zabezpečení činnosti rezortu Ministe...,True


In [38]:
# create a UDPipe processing pipeline with the loaded model,
# with "horizontal" input (a sentence with space-separated tokens),
# default setting for tagger and parser,
# and CoNLL-U output
udp_pipeline = Pipeline(udpipe_model, "tokenize", Pipeline.DEFAULT, Pipeline.DEFAULT, "conllu")

In [39]:
%%time

decomposition = udp_pipeline.process(text)
print(type(decomposition))
# print(decomposition)

NameError: name 'text' is not defined

In [204]:
forest = conllu.parse(decomposition)
forest

[TokenList<ajů, oznámí, bez, prodlení, druhé, smluvní, straně, .>,
 TokenList<Strany, dále, prohlašují, ,, že, osoby, podepisující, smlouvu, jsou, k, tomuto, úkonu, oprávněny, ,>,
 TokenList<Zhotovitel, je, odborně, způsobilý, k, zajištění, předmětu, plnění, podle, smlouvy, .>,
 TokenList<Zhotovitel, prohlašuje, ,, že, po, celou, dobu, platnosti, smlouvy, bude, mít, sjednánu, pojistnou, smlouvu, pro, případ, způsobení, škody, ve, výši, 20, milionů, korun, českých, ,, požadované, zadavatelem, ,>,
 TokenList<Smlouva, nabývá, platnosti, a, účinnosti, dnem, podpisu, obou, smluvních, stran, ,>,
 TokenList<II, .>,
 TokenList<PŘEDMĚT, SMLOUVY>,
 TokenList<Zhotovitel, se, zavazuje, k, provedení, stavebních, prací, v, rámci, projektu, „, Stezka, pro, cyklisty, -, k, ., ú, .>,
 TokenList<Starý, Mateřov“, (, dále, jen, „, veřejná, zakázka“, ), .>,
 TokenList<Objednatel, se, zavazuje, předmět, smlouvy, převzít, bez, vad, a, nedodělků, v, době, předání, a, zaplatit, za, ně, zhotoviteli, cenu, podle

In [186]:
if tree.filter(form='bez'):
    print('ahoj')

ahoj


In [212]:
for tree in forest:
    if tree.filter(lemma='zhotovitel'):
        print(' '.join([token['form'] for token in tree]))

Vlastníkem zařízení staveniště , včetně používaných strojů , mechanismů a dalších věcí potřebných pro provedení díla , je zhotovitel , který nese nebezpečí škody na těchto věcech .
Za škody vzniklé na prováděném díle nese zodpovědnost až do převzetí díla objednatelem zhotovitel , Veškeré náklady vzniklé v souvislosti s odstraňováním škod nese zhotovitel a tyto náklady nemají vliv na sjednanou cenu díla .


In [122]:
text='Máma mele maso. Táta vaří brambory.'

In [131]:
root = conllu.parse_tree(udp_pipeline.process(text))[1]
for node in root.children:
    node.print_tree()

(deprel:nsubj) form:Táta lemma:táta upostag:NOUN [1]
(deprel:obj) form:brambory lemma:brambora upostag:NOUN [3]
(deprel:punct) form:. lemma:. upostag:PUNCT [4]


In [46]:
pandas.DataFrame()

ValueError: DataFrame constructor not properly called!